In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
#Create local file to avoid issue with library
from pathlib import Path  

In [3]:
#Got airports from table flights
df_airports = pd.read_csv("../../data/raw/airportsFlightsTable.csv")
df_airports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376 entries, 0 to 375
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   airport_id  376 non-null    int64 
 1   iata_code   376 non-null    object
 2   city        376 non-null    object
dtypes: int64(1), object(2)
memory usage: 8.9+ KB


In [4]:
#Got airports from https://ourairports.com/airports.html
global_airports = pd.read_csv("../../data/raw/worldAirports.csv")

In [5]:
df_global_airports = global_airports[['ident','iata_code','latitude_deg','longitude_deg']]
df_global_airports.rename(columns={'latitude_deg': 'lat','longitude_deg':'lon'}, inplace = True)
df_global_airports

,ident,iata_code,lat,lon
0,EGLL,LHR,51.470600,-0.461941
1,KLAX,LAX,33.942501,-118.407997
2,KORD,ORD,41.978600,-87.904800
3,KJFK,JFK,40.639447,-73.779317
4,KATL,ATL,33.636700,-84.428101
...,...,...,...,...
35484,CMB7,NaN,45.253000,-74.807000
35485,12CN,NaN,39.268306,-122.116689
35486,8CL6,NaN,39.288502,-122.189003
35487,0ID2,NaN,44.442902,-113.773003


In [6]:
df_airports_merged = df_airports.merge(df_global_airports, on = 'iata_code', how = 'inner' )
df_airports_merged = df_airports_merged.drop(columns=['city','ident'])
df_airports_merged['Key'] = df_airports_merged['lat'].astype(str) +'-'+df_airports_merged['lat'].astype(str)
df_airports_merged


,airport_id,iata_code,lat,lon,Key
0,10135,ABE,40.651773,-75.442797,40.651773-40.651773
1,10136,ABI,32.411301,-99.681900,32.4113006592-32.4113006592
2,10140,ABQ,35.040199,-106.609001,35.040199-35.040199
3,10141,ABR,45.449100,-98.421799,45.449100494384766-45.449100494384766
4,10146,ABY,31.532946,-84.196215,31.532946-31.532946
...,...,...,...,...,...
369,15991,YAK,59.508717,-139.660435,59.508717-59.508717
370,16101,YKM,46.568199,-120.543999,46.56819916-46.56819916
371,16133,YNG,41.260700,-80.679100,41.26070023-41.26070023
372,16218,YUM,32.656601,-114.606003,32.656601-32.656601


In [8]:
#Create local file for airports id, iatacode, latitute and longitude
from pathlib import Path  
filepath = Path('../../data/raw/Airports_lat_lon.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_airports_merged.to_csv(filepath)  

In [10]:
pip install meteostat

Note: you may need to restart the kernel to use updated packages.


In [11]:
#Import meteostast library and datetime
from datetime import datetime
from meteostat import Point, Daily

start = datetime(2018, 1, 1) # Change start date
end = datetime(2020, 1, 31) # Change end date

In [ ]:
#use weather library meteostat
#Be carefull this library is not availble all the time and generate connection errors
#def get_bulk_weather(df):
#    weather_dfs=[]
#    for i,r in df.iterrows():
#        point = Point(r.lat, r.lon)
#        data = Daily(point, start, end)
#        data = data.fetch()
#        data["lat"] = r.lat
#        data["lon"] = r.lon
#        weather_dfs.append(data)
#    weather_final = pd.concat(weather_dfs)
#    return weather_final 
#
#df_weather = get_bulk_weather(df_airports_merged)
#df_weather

In [ ]:

#filepath = Path('/Users/hugo/Library/CloudStorage/OneDrive-Personal/DATA_Science/Bootcamp/W7/mid-term-project-I/data/raw/weatherAirports_2019_2018_Jan2020.csv')  
#filepath.parent.mkdir(parents=True, exist_ok=True)  
#df_weather.to_csv(filepath)  

In [13]:
df_weather_f = pd.read_csv("../../data/processed/weatherAirports_2019_2018_Jan2020_clean.csv")  

In [14]:
df_weather_f

,Unnamed: 0,fl_date,AvgTemp,Precipitation,Snow_mm,WindSpeed,lat,lon,Key
0,0,2018-01-01,-11.7,0.0,30.0,13.0,40.651773,-75.442797,40.651773-40.651773
1,1,2018-01-02,-9.4,0.0,30.0,17.3,40.651773,-75.442797,40.651773-40.651773
2,2,2018-01-03,-8.9,0.0,0.0,4.3,40.651773,-75.442797,40.651773-40.651773
3,3,2018-01-04,-5.0,2.5,0.0,24.5,40.651773,-75.442797,40.651773-40.651773
4,4,2018-01-05,-10.8,0.0,30.0,29.9,40.651773,-75.442797,40.651773-40.651773
...,...,...,...,...,...,...,...,...,...
271661,271661,2019-12-25,0.0,0.0,0.0,0.0,48.258387,-103.748797,48.258387-48.258387
271662,271662,2019-12-26,0.0,0.0,0.0,0.0,48.258387,-103.748797,48.258387-48.258387
271663,271663,2019-12-27,0.0,0.0,0.0,0.0,48.258387,-103.748797,48.258387-48.258387
271664,271664,2019-12-28,0.0,0.0,0.0,0.0,48.258387,-103.748797,48.258387-48.258387


In [16]:
# df_weather_f = df_weather_f.drop(columns=['tmin','tmax','wdir','pres','wpgt','tsun']) 
df_weather_f.rename(columns={'Unnamed: 0' :'fl_date','tavg':'AvgTemp', 'prcp':'Precipitation', 'snow':'Snow_mm','wspd':'WindSpeed'}, inplace=True)
df_weather_f['Key'] = df_weather_f['lat'].astype(str) +'-'+df_weather_f['lat'].astype(str)


In [17]:
df_weather_f.fillna(0, inplace=True)
#df_weather_f.drop_duplicates(inplace=True)
df_weather_f

,fl_date,fl_date,AvgTemp,Precipitation,Snow_mm,WindSpeed,lat,lon,Key
0,0,2018-01-01,-11.7,0.0,30.0,13.0,40.651773,-75.442797,40.651773-40.651773
1,1,2018-01-02,-9.4,0.0,30.0,17.3,40.651773,-75.442797,40.651773-40.651773
2,2,2018-01-03,-8.9,0.0,0.0,4.3,40.651773,-75.442797,40.651773-40.651773
3,3,2018-01-04,-5.0,2.5,0.0,24.5,40.651773,-75.442797,40.651773-40.651773
4,4,2018-01-05,-10.8,0.0,30.0,29.9,40.651773,-75.442797,40.651773-40.651773
...,...,...,...,...,...,...,...,...,...
271661,271661,2019-12-25,0.0,0.0,0.0,0.0,48.258387,-103.748797,48.258387-48.258387
271662,271662,2019-12-26,0.0,0.0,0.0,0.0,48.258387,-103.748797,48.258387-48.258387
271663,271663,2019-12-27,0.0,0.0,0.0,0.0,48.258387,-103.748797,48.258387-48.258387
271664,271664,2019-12-28,0.0,0.0,0.0,0.0,48.258387,-103.748797,48.258387-48.258387


In [ ]:
#filepath = Path('/Users/hugo/Library/CloudStorage/OneDrive-Personal/DATA_Science/Bootcamp/W7/mid-term-project-I/data/processed/weatherAirports_2019_2018_Jan2020_clean.csv')  
#filepath.parent.mkdir(parents=True, exist_ok=True)  
#df_weather_f.to_csv(filepath)  

In [73]:
from add_location_weather import add_location
from add_location_weather import add_weather

In [74]:
weather_airports = pd.read_csv("../../data/processed/weatherAirports_2019_2018_Jan2020_clean.csv")
df_week_one  = pd.read_csv("../../data/raw/Week_one_data_10000.csv")
airports_ids =  pd.read_csv("../../data/processed/Airports_lat_lon.csv")


In [101]:
df = pd.read_csv("random_sample_flight_data.csv")
df['fl_date'] = df['fl_date'].astype('datetime64')
df

,op_carrier_fl_num,op_unique_carrier,origin_airport_id,dest_airport_id,distance,route,month,day,fl_date,arr_delay
0,1610,UA,11618,13495,1167,11618-13495,3,1,2019-03-01,0.0
1,1519,UA,10800,11292,850,10800-11292,7,19,2019-07-19,-5.0
2,5452,OO,14487,14771,199,14487-14771,6,25,2018-06-25,119.0
3,1054,B6,14027,12478,1028,14027-12478,11,30,2018-11-30,3.0
4,790,UA,13930,10721,867,13930-10721,3,26,2019-03-26,-8.0
...,...,...,...,...,...,...,...,...,...,...
99995,6279,YV,12266,10994,925,12266-10994,7,31,2018-07-31,226.0
99996,2047,UA,12266,10721,1597,12266-10721,12,31,2018-12-31,-37.0
99997,7406,OO,11433,13459,349,11433-13459,7,22,2018-07-22,-22.0
99998,5151,OH,12323,11057,185,12323-11057,3,5,2019-03-05,109.0


In [102]:
# airports_ids = airports_ids.drop(['Unnamed: 0'], axis=1)
airports_ids

,airport_id,iata_code,lat,lon,Key
0,10135,ABE,40.651773,-75.442797,40.651773-40.651773
1,10136,ABI,32.411301,-99.681900,32.4113006592-32.4113006592
2,10140,ABQ,35.040199,-106.609001,35.040199-35.040199
3,10141,ABR,45.449100,-98.421799,45.449100494384766-45.449100494384766
4,10146,ABY,31.532946,-84.196215,31.532946-31.532946
...,...,...,...,...,...
369,15991,YAK,59.508717,-139.660435,59.508717-59.508717
370,16101,YKM,46.568199,-120.543999,46.56819916-46.56819916
371,16133,YNG,41.260700,-80.679100,41.26070023-41.26070023
372,16218,YUM,32.656601,-114.606003,32.656601-32.656601


In [103]:
df1 = add_location(df,airports_ids)
df1 = df1.drop(['airport_id','iata_code','lat','lon'], axis=1)
df1

,op_carrier_fl_num,op_unique_carrier,origin_airport_id,dest_airport_id,distance,route,month,day,fl_date,arr_delay,Key
0,1610,UA,11618,13495,1167,11618-13495,3,1,2019-03-01,0.0,40.692501-40.692501
1,4909,C5,11618,14524,277,11618-14524,12,6,2019-12-06,-17.0,40.692501-40.692501
2,3556,YX,11618,11193,569,11618-11193,4,3,2018-04-03,-9.0,40.692501-40.692501
3,1382,AA,11618,11298,1372,11618-11298,9,7,2019-09-07,68.0,40.692501-40.692501
4,3517,YX,11618,13495,1167,11618-13495,2,5,2018-02-05,-37.0,40.692501-40.692501
...,...,...,...,...,...,...,...,...,...,...,...
99966,5026,OO,11092,11292,283,11092-11292,9,22,2019-09-22,0.0,38.75500107-38.75500107
99967,5031,OO,11092,11292,283,11092-11292,8,10,2019-08-10,-12.0,38.75500107-38.75500107
99968,5079,OO,11092,11292,283,11092-11292,8,15,2018-08-15,2.0,38.75500107-38.75500107
99969,5031,OO,11092,11292,283,11092-11292,5,26,2019-05-26,-22.0,38.75500107-38.75500107


In [104]:
# weather_airports = weather_airports.drop(['Unnamed: 0'], axis=1)
weather_airports['fl_date'] = weather_airports['fl_date'].astype('datetime64')
weather_airports

,fl_date,AvgTemp,Precipitation,Snow_mm,WindSpeed,lat,lon,Key
0,2018-01-01,-11.7,0.0,30.0,13.0,40.651773,-75.442797,40.651773-40.651773
1,2018-01-02,-9.4,0.0,30.0,17.3,40.651773,-75.442797,40.651773-40.651773
2,2018-01-03,-8.9,0.0,0.0,4.3,40.651773,-75.442797,40.651773-40.651773
3,2018-01-04,-5.0,2.5,0.0,24.5,40.651773,-75.442797,40.651773-40.651773
4,2018-01-05,-10.8,0.0,30.0,29.9,40.651773,-75.442797,40.651773-40.651773
...,...,...,...,...,...,...,...,...
271661,2019-12-25,0.0,0.0,0.0,0.0,48.258387,-103.748797,48.258387-48.258387
271662,2019-12-26,0.0,0.0,0.0,0.0,48.258387,-103.748797,48.258387-48.258387
271663,2019-12-27,0.0,0.0,0.0,0.0,48.258387,-103.748797,48.258387-48.258387
271664,2019-12-28,0.0,0.0,0.0,0.0,48.258387,-103.748797,48.258387-48.258387


In [105]:
# df2= add_weather(df1,weather_airports)
# df2

df2 = pd.merge(df1,weather_airports, on = ['Key','fl_date'], how ='left')
results.drop_duplicates(inplace=True)
df2

,op_carrier_fl_num,op_unique_carrier,origin_airport_id,dest_airport_id,distance,route,month,day,fl_date,arr_delay,Key,AvgTemp,Precipitation,Snow_mm,WindSpeed,lat,lon
0,1610,UA,11618,13495,1167,11618-13495,3,1,2019-03-01,0.0,40.692501-40.692501,0.8,1.3,30.0,15.9,40.692501,-74.168701
1,4909,C5,11618,14524,277,11618-14524,12,6,2019-12-06,-17.0,40.692501-40.692501,5.7,0.0,0.0,12.2,40.692501,-74.168701
2,3556,YX,11618,11193,569,11618-11193,4,3,2018-04-03,-9.0,40.692501-40.692501,4.9,5.8,30.0,11.1,40.692501,-74.168701
3,1382,AA,11618,11298,1372,11618-11298,9,7,2019-09-07,68.0,40.692501-40.692501,20.4,0.0,0.0,6.3,40.692501,-74.168701
4,3517,YX,11618,13495,1167,11618-13495,2,5,2018-02-05,-37.0,40.692501-40.692501,0.3,0.0,0.0,13.4,40.692501,-74.168701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99966,5026,OO,11092,11292,283,11092-11292,9,22,2019-09-22,0.0,38.75500107-38.75500107,16.0,0.0,0.0,0.0,38.755001,-109.754997
99967,5031,OO,11092,11292,283,11092-11292,8,10,2019-08-10,-12.0,38.75500107-38.75500107,27.3,0.0,0.0,0.0,38.755001,-109.754997
99968,5079,OO,11092,11292,283,11092-11292,8,15,2018-08-15,2.0,38.75500107-38.75500107,28.5,0.0,0.0,10.7,38.755001,-109.754997
99969,5031,OO,11092,11292,283,11092-11292,5,26,2019-05-26,-22.0,38.75500107-38.75500107,18.3,0.0,0.0,11.4,38.755001,-109.754997


In [106]:
df2.columns.tolist()

['op_carrier_fl_num',
 'op_unique_carrier',
 'origin_airport_id',
 'dest_airport_id',
 'distance',
 'route',
 'month',
 'day',
 'fl_date',
 'arr_delay',
 'Key',
 'AvgTemp',
 'Precipitation',
 'Snow_mm',
 'WindSpeed',
 'lat',
 'lon']

In [108]:
# drop rows with missing values
df2 = df2.dropna().reset_index(drop=True)
df2.shape

(97307, 17)

In [109]:
df2.to_csv('transformed_random_flight_data.csv', index=False)

## Week One Transformed Test Data

In [20]:
df_week_one

,op_carrier_fl_num,op_unique_carrier,origin_airport_id,dest_airport_id,distance,route,day
0,5390,WN,14057,11292,991.0,14057-11292,1
1,2331,WN,15376,11292,639.0,15376-11292,2
2,2321,NK,14843,11618,1608.0,14843-11618,5
3,5496,OO,11109,12266,809.0,11109-12266,3
4,2811,QX,11884,14057,279.0,11884-14057,3
...,...,...,...,...,...,...,...
9995,1447,DL,12953,10397,762.0,12953-10397,2
9996,167,NK,10821,14492,255.0,10821-14492,4
9997,1037,UA,11618,11298,1372.0,11618-11298,7
9998,6130,YX,14122,12953,335.0,14122-12953,3


In [38]:
results1 = add_location(df_week_one,airports_ids)
results1


,op_carrier_fl_num,op_unique_carrier,origin_airport_id,dest_airport_id,distance,route,day,airport_id,iata_code,lat,lon,Key
0,5390,WN,14057,11292,991.0,14057-11292,1,14057,PDX,45.588699,-122.598000,45.58869934-45.58869934
1,5752,CP,14057,14747,129.0,14057-14747,7,14057,PDX,45.588699,-122.598000,45.58869934-45.58869934
2,789,UA,14057,11292,991.0,14057-11292,2,14057,PDX,45.588699,-122.598000,45.58869934-45.58869934
3,1184,AS,14057,14679,933.0,14057-14679,7,14057,PDX,45.588699,-122.598000,45.58869934-45.58869934
4,1552,AS,14057,14262,873.0,14057-14262,7,14057,PDX,45.588699,-122.598000,45.58869934-45.58869934
...,...,...,...,...,...,...,...,...,...,...,...,...
9994,5147,OO,14006,13930,342.0,14006-13930,6,14006,PAH,37.060799,-88.773804,37.06079864501953-37.06079864501953
9995,5081,OO,13121,13930,489.0,13121-13930,2,13121,LWB,37.858299,-80.399498,37.8582992554-37.8582992554
9996,3845,MQ,11905,11298,140.0,11905-11298,2,11905,GGG,32.383999,-94.711502,32.38399887084961-32.38399887084961
9997,4258,OO,12003,14869,463.0,12003-14869,6,12003,GTF,47.481998,-111.371002,47.48199844-47.48199844


In [40]:
results= add_weather(results1,weather_airports)

In [41]:
results

,op_carrier_fl_num,op_unique_carrier,origin_airport_id,dest_airport_id,distance,route,day,airport_id,iata_code,lat_x,lon_x,Key,fl_date,AvgTemp,Precipitation,Snow_mm,WindSpeed,lat_y,lon_y
0,5390,WN,14057,11292,991.0,14057-11292,1,14057,PDX,45.588699,-122.598000,45.58869934-45.58869934,2020-01-01,10.8,3.8,0.0,11.9,45.588699,-122.598000
1,5752,CP,14057,14747,129.0,14057-14747,7,14057,PDX,45.588699,-122.598000,45.58869934-45.58869934,2020-01-07,11.3,2.8,0.0,24.8,45.588699,-122.598000
2,789,UA,14057,11292,991.0,14057-11292,2,14057,PDX,45.588699,-122.598000,45.58869934-45.58869934,2020-01-02,8.2,1.0,0.0,11.9,45.588699,-122.598000
3,1184,AS,14057,14679,933.0,14057-14679,7,14057,PDX,45.588699,-122.598000,45.58869934-45.58869934,2020-01-07,11.3,2.8,0.0,24.8,45.588699,-122.598000
4,1552,AS,14057,14262,873.0,14057-14262,7,14057,PDX,45.588699,-122.598000,45.58869934-45.58869934,2020-01-07,11.3,2.8,0.0,24.8,45.588699,-122.598000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,5147,OO,14006,13930,342.0,14006-13930,6,14006,PAH,37.060799,-88.773804,37.06079864501953-37.06079864501953,2020-01-06,5.3,0.0,0.0,7.9,37.060799,-88.773804
9995,5081,OO,13121,13930,489.0,13121-13930,2,13121,LWB,37.858299,-80.399498,37.8582992554-37.8582992554,2020-01-02,1.9,0.0,0.0,1.8,37.858299,-80.399498
9996,3845,MQ,11905,11298,140.0,11905-11298,2,11905,GGG,32.383999,-94.711502,32.38399887084961-32.38399887084961,2020-01-02,12.1,4.6,0.0,11.9,32.383999,-94.711502
9997,4258,OO,12003,14869,463.0,12003-14869,6,12003,GTF,47.481998,-111.371002,47.48199844-47.48199844,2020-01-06,0.1,0.0,0.0,26.3,47.481998,-111.371002


In [42]:
results.to_csv('transformed_week1_2020.csv', index=False)